goal of this notebook: introduce this project and its results

(what is k nearest p median and how to achieve it)

towards the problem, we need to define things like:
a. what type of input data belongs to? (from_cost_matrix/from_geodataframe/cost_dataframe. if from_geodataframe, the way to query the nearest facilities in spatial area)
b. how to deal with placeholder facility? (decision variable for them, how it works. it considers the impact of far facility in a special way, no need to store the large data of them, but still indicates if the client needs them)
c. how to deal with large na values enrousing from k nearest facilities? (normal matrix → sparse matrix. plus the latter has advantages like reducing storage space)

moreover, at the level of programming design, we need to decide things like:
a. what methods we need in this new class? (update k_list, create_sparse_matrix, solve, etc...)
b. what are the parameters and attributes of the object, and how to use them during the whole process? (the model becomes one of the attributes of the object, we call it 'problem'. and each loop/iteration will refresh/update the 'problem', the k_list and the sparse matrix)
c. what part we want to show to user, which part the user don't need to know? (we want the user to get access to the latest k_list besides the normal model results. and some methods, like, update k_list, create_sparse_matrix, the user don't need to know and use)

# K Nearest P Median Problem and Implementation

## Introduction

The P-Median Model with Near-Far Cost Allocation, alternatively known as the k nearest p-median Problem, offers a distinctive twist on the traditional p-median problem. This approach enhances our capability to effectively address location allocation challenges. In this notebook, we'll dive into the problem's formulation, and showcase the implementation of the `KNearestPMedian` class in the `spopt` package.

### Understanding the K Nearest P-median Problem

The k nearest p-median problem extends the concept of facility location allocation by considering both nearest and non-nearest facilities. In the article by Church (2018), it proposed this new p-median model which can distinguish between near and far facilities, use both explicit and implicit variables for capacity allocations. By integrating this model, the `spopt` package becomes equipped to deliver heightened precision and efficiency in solving spatial optimization problems, especially those involving extensive datasets and substantial demand volumes.

The model can be formulated as:

$\begin{array}
       \displaystyle \textbf{Minimize} & \displaystyle \sum_{i \in I}\sum_{k \in k_{i}}{a_i d_{ik} X_{ik}} + \sum_{i \in I}{g_i (d_{i{k_i}} + 1)}  &&& (1) \\
       \displaystyle \textbf{Subject To} & \sum_{k \in k_{i}}{X_{ik} + g_i = 1} && \forall i \in I & (2) \\
                                            & \sum_{j \in J}{Y_j} = p                                                                                   &&                                          & (3)                                                                               \\
                                            & \sum_{i \in I}{a_i X_{ik}} \leq {Y_{k} c_{k}}                                                             &&  \forall k \in k_{i}                     & (4)                                                                               \\  
                                            & X_{ij} \leq Y_{j}                                                                                         && \forall i \in I \quad \forall j \in J    & (5)                                                                               \\
                                            & X_{ij} \in \{0, 1\}                                                                                       && \forall i \in I \quad \forall j \in J    & (6)                                                                               \\
                                            & Y_j \in \{0, 1\}                                                                                          && \forall j \in J                          & (7)                                                                               \\
                                            &                                                                                                           &&                                          &                                                                                   \\ \end{array}$
                                            
$\begin{array} \displaystyle \textbf{Where}&& i& =& \textrm{index of demand points/areas/objects in set } I\\
                                            && j                                                                                                        & =                                         & \textrm{index of potential facility sites in set } J                              \\
                                            && p                                                                                                        & =                                         & \textrm{the number of facilities to be sited}                                     \\
                                            && a_i                                                                                                      & =                                         & \textrm{service load or population demand at client location } i                  \\
                                            && k_{i}                                                                                                    & =                                         & \textrm{the } k \textrm{ nearest facilities of client location } i                        \\
                                            && c_{j}                                                                                                    & =                                         & \textrm{the capacity of facility} j                                               \\   
                                            && d_{ij}                                                                                                   & =                                         & \textrm{shortest distance or travel time between locations } i \textrm{ and } j   \\
                                            && X_{ij}                                                                                                   & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if client location } i \textrm{ is served by facility } j             \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\
                                            && Y_j                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if a facility is sited at location } j                                \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\ 
                                            && g_i                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if the client } i \textrm{ need to be served by non-k-nearest facilities}     \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\ \end{array}$

This model introduces a clever approach by differentiating between nearby and distant facilities. For instance, only the five closest facilities to each client are considered, which are termed as "near" facilities. If all client demands and constraints can be satisfied, the problem will have an optimal solution right away. However, if the initial facilities can't meet all the demands, we can supplement them with additional nearby facilities.

Mathematically, we introduce a variable called the placeholder facility decision variable, which is $g_i$
in the formula, presenting the distant facilities.

In terms of the solution approach:
1. For each client, identify their five closest facilities.
2. Run the p-median model only for the selected facilities from Step 1 (referred to as "explicitly assigned" in the paper).
3. Check the model outcome. If all $g_i$ values for clients are equal to 0 and an optimal solution exists, we accept this solution.
4. If some $g_i$ values are more than 0, we need to increase the value $k$ for specific clients. Then, rerun the model. Ensure that all $g_i$ values become 0, indicating an optimal solution.

The default starting value for $k$ is 5, or the total number of facilities if it is fewer than 5. Nevertheless, users have the flexibility to customize this value according to their particular requirements.

## The `KNearestPMedian` Class Implementation

### Input Data Type: Geodataframe

To cater to diverse scenarios, the `KNearestPMedian` class supports one input data type:

1. **From GeoDataFrame:** Leverage geospatial data for dynamic distance calculations based on a chosen metric.

### Efficiency with Sparse Matrices

Managing large datasets efficiently is paramount. The `KNearestPMedian` class employs sparse matrices to optimize memory usage while retaining computational efficiency for distance calculations.

The `KNearestPMedian` class and its key methods are as following:


In [ ]:
class KNearestPMedian(PMedian):
    r"""
    Implement the P-Median Model with Near-Far Cost Allocation and solve it. 
    The model is adapted from :cite:`richard_2018`, can be formulated as:

    .. math::

       \begin{array}{lllll}
       \displaystyle \textbf{Minimize}      & \displaystyle \sum_{i \in I}\sum_{k \in k_{i}}{a_i d_{ik} X_{ik}} + \sum_{i \in I}{g_i (d_{i{k_i}} + 1)}  &&                                          & (1)                                                                               \\
       \displaystyle \textbf{Subject To}    & \sum_{k \in k_{i}}{X_{ik} + g_i = 1}                                                                     && \forall i \in I                          & (2)                                                                               \\
                                            & \sum_{j \in J}{Y_j} = p                                                                                   &&                                          & (3)                                                                               \\
                                            & \sum_{i \in I}{a_i X_{ik}} \leq {Y_{k} c_{k}}                                                             &&  \forall k \in k_{i}                     & (4)                                                                               \\  
                                            & X_{ij} \leq Y_{j}                                                                                         && \forall i \in I \quad \forall j \in J    & (5)                                                                               \\
                                            & X_{ij} \in \{0, 1\}                                                                                       && \forall i \in I \quad \forall j \in J    & (6)                                                                               \\
                                            & Y_j \in \{0, 1\}                                                                                          && \forall j \in J                          & (7)                                                                               \\
                                            &                                                                                                           &&                                          &                                                                                   \\
       \displaystyle \textbf{Where}         && i                                                                                                        & =                                         & \textrm{index of demand points/areas/objects in set } I                           \\
                                            && j                                                                                                        & =                                         & \textrm{index of potential facility sites in set } J                              \\
                                            && p                                                                                                        & =                                         & \textrm{the number of facilities to be sited}                                     \\
                                            && a_i                                                                                                      & =                                         & \textrm{service load or population demand at client location } i                  \\
                                            && k_{i}                                                                                                    & =                                         & \textrm{the } k \textrm{nearest facilities of client location } i                 \\
                                            && c_{j}                                                                                                    & =                                         & \textrm{the capacity of facility} j                                               \\   
                                            && d_{ij}                                                                                                   & =                                         & \textrm{shortest distance or travel time between locations } i \textrm{ and } j   \\
                                            && X_{ij}                                                                                                   & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if client location } i \textrm{ is served by facility } j             \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\
                                            && Y_j                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if a facility is sited at location } j                                \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\ 
                                            && g_i                                                                                                      & =                                         & \begin{cases}
                                                                                                                                                                                                       1, \textrm{if the client } i \textrm{ needs to be served by non-k-nearest facilities}     \\
                                                                                                                                                                                                       0, \textrm{otherwise}                                                            \\
                                                                                                                                                                                                      \end{cases}                                                                       \\
       \end{array}

    Parameters
    ----------

    name : str
        The problem name.
    ai_sum : Union[int, float]
        The sum of weights representing the service loads of the clients.
    clients : np.array
        An array of coordinates of clients.
    facilities : np.array
        An array of coordinates of facilities.
    weights : np.array
        An array of weights representing the service loads of the clients.
    p_facilities: int
        The number of facilities to be located.
    capacities : np.array or None
        An array of facility capacities. None if capacity constraints are
        not considered.
    k_array : np.array
        An array of k values representing the number of nearest facilities
        for each client.
    distance_metric : str
        The distance metric used for computing distances between clients
        and facilities.
    
    Attributes
    ----------

    sparse_matrix : Compressed Sparse Row matrix
        A cost matrix in the form of a compressed sparse row matrix between
        origins and destinations.
    aij : Compressed Sparse Row matrix
        A weighted cost matrix in the form of a compressed sparse row matrix
        between origins and destinations.
    problem : pulp.LpProblem
        A ``pulp`` instance of an optimization model that contains constraints,
        variables, and an objective function.
    fac2cli : numpy.array
        A 2D array storing facility to client relationships where each
        row represents a facility and contains an array of client indices
        with which it is associated. An empty client array indicates
        the facility is associated with no clients.
    cli2fac : numpy.array
        The inverse of ``fac2cli`` where client to facility relationships
        are shown.

    """  # noqa

    def __init__(
        self,
        weights_sum: Union[int, float],
        clients: np.array,
        facilities: np.array,
        weights: np.array,
        k_array: np.array,
        p_facilities: int,
        capacities: np.array = None,
        distance_metric: str = "euclidean",
        name="k-nearest p median",
    ):
        self.ai_sum = weights_sum
        self.clients = clients
        self.facilities = facilities
        self.weights = weights
        self.k_array = k_array
        self.p_facilities = p_facilities
        self.capacities = capacities
        self.distance_metric = distance_metric
        self.name = name

    def __add_obj(
        self, max_distance: np.array, range_clients: range, range_facility: range
    ) -> None:
        """
        Add the objective function to the model.

        Parameters
        ----------

        max_distance : np.array
            An array of distances between each client and their k nearest facility.
            For example, when k = 2, this array will only store the distances between
            each client and their nearest and second nearest facility.
        range_clients : range
            The range of demand points.
        range_facility : range
            The range of facility point.

        Returns
        -------

        None

        """
        cli_assgn_vars = getattr(self, "cli_assgn_vars")
        placeholder_vars = getattr(self, "placeholder_vars")

        self.problem += (
            pulp.lpSum(
                pulp.lpSum(
                    self.aij[i, j] * cli_assgn_vars.get((i, j), 0)
                    for j in range_facility
                )
                + (placeholder_vars[i] * (max_distance[i] + 1))
                for i in range_clients
            ),
            "objective function",
        )

    @classmethod
    def from_cost_matrix(cls, *args, **kwargs):
        """
        Warning: This method is not supported in the KNearestPMedian subclass.
        """
        raise NotImplementedError(
            "from_cost_matrix method is not supported in KNearestPMedian class."
        )

    def _create_sparse_matrix(self) -> None:
        """
        Create a sparse matrix representing the distance between clients
        and their k nearest facilities.

        This method uses a suitable tree data structure (built with the
        `build_best_tree` function) to efficiently find the k nearest
        facilities for each client based on the specified distance metric.
        The resulting distances are stored in a sparse matrix format to
        conserve memory for large datasets.

        Returns
        -------

        None
        """

        row_shape = len(self.clients)
        column_shape = len(self.facilities)

        # check the k value with the total number of facilities
        if not (self.k_array <= column_shape).all():
            raise ValueError(
                f"The value of k should be no more than the number of total"
                f"facilities ({column_shape})."
            )

        # Initialize empty lists to store the data for the sparse matrix
        data = []
        row_index = []
        col_index = []

        # create the suitable Tree
        tree = build_best_tree(self.facilities, self.distance_metric)

        for i, k in enumerate(self.k_array):
            # Query the Tree to find the k nearest facilities for each client
            distances, k_nearest_facilities_indices = tree.query([self.clients[i]], k=k)

            # extract the contents of the inner array
            distances = distances[0].tolist()
            k_nearest_facilities_indices = k_nearest_facilities_indices[0].tolist()

            # Append the data for the sparse matrix
            data.extend(distances)
            row_index.extend([i] * k)
            col_index.extend(k_nearest_facilities_indices)

        # Create the sparse matrix using csr_matrix
        self.sparse_matrix = csr_matrix(
            (data, (row_index, col_index)), shape=(row_shape, column_shape)
        )

    def _update_k_array(self) -> None:
        """
        Increase the k value for clients with any g_i > 0 and update the k array.

        This method is used to adjust the k values for clients based on their
        placeholder variable g_i. For clients with g_i greater than 0, the
        corresponding k value is increased by 1 in the new k array.

        Returns
        -------

        None
        """

        new_k_array = self.k_array.copy()
        placeholder_vars = getattr(self, "placeholder_vars")
        for i in range(len(placeholder_vars)):
            if placeholder_vars[i].value() > 0:
                new_k_array[i] = new_k_array[i] + 1
        self.k_array = new_k_array

    def _from_sparse_matrix(self) -> None:
        """
        Create the k nearest p-median problem from the sparse distance matrix.

        Returns
        -------

        None
        """
        n_cli = self.sparse_matrix.shape[0]
        r_cli = range(n_cli)
        r_fac = range(self.sparse_matrix.shape[1])

        self.weights = np.reshape(self.weights, (n_cli, 1))
        # create and store the weighted sparse matrix
        self.aij = self.sparse_matrix.multiply(self.weights).tocsr()

        # create the model/problem
        self.problem = pulp.LpProblem(self.name, pulp.LpMinimize)

        # add all the 1)decision variable, 2)objective function, and 3)constraints

        # Facility integer decision variable
        FacilityModelBuilder.add_facility_integer_variable(self, r_fac, "y[{i}]")
        fac_vars = getattr(self, "fac_vars")

        # Placeholder facility decision variable
        placeholder_vars = pulp.LpVariable.dicts(
            "g", (i for i in r_cli), 0, 1, pulp.LpBinary
        )
        setattr(self, "placeholder_vars", placeholder_vars)

        # Client assignment integer decision variables
        row_indices, col_indices, values = find(self.aij)
        cli_assgn_vars = pulp.LpVariable.dicts(
            "z", [(i, j) for i, j in zip(row_indices, col_indices)], 0, 1, pulp.LpBinary
        )
        setattr(self, "cli_assgn_vars", cli_assgn_vars)

        # Add the objective function
        max_distance = self.aij.max(axis=1).toarray().flatten()
        self.__add_obj(max_distance, r_cli, r_fac)

        # Create the capacity constraints
        if self.capacities is not None:
            sorted_capacities = np.sort(self.capacities)
            highest_possible_capacity = sorted_capacities[-self.p_facilities :].sum()
            if highest_possible_capacity < self.ai_sum:
                raise SpecificationError(
                    "Problem is infeasible. The highest possible capacity "
                    f"{highest_possible_capacity}, coming from the {self.p_facilities} "
                    "sites with the highest capacity, is smaller than "
                    f"the total demand {self.ai_sum}."
                )
            for j in col_indices:
                self.problem += (
                    pulp.lpSum(
                        self.weights[i] * cli_assgn_vars.get((i, j), 0) for i in r_cli
                    )
                    <= fac_vars[j] * self.capacities[j]
                )

        # Create assignment constraints.
        for i in r_cli:
            self.problem += (
                pulp.lpSum(cli_assgn_vars.get((i, j), 0) for j in set(col_indices))
                + placeholder_vars[i]
                == 1
            )
        # Create the facility constraint.
        FacilityModelBuilder.add_facility_constraint(self, self.p_facilities)

    @classmethod
    def from_geodataframe(
        cls,
        gdf_demand: GeoDataFrame,
        gdf_fac: GeoDataFrame,
        demand_col: str,
        facility_col: str,
        weights_cols: str,
        p_facilities: int,
        facility_capacity_col: str = None,
        k_array: np.array = None,
        distance_metric: str = "euclidean",
        name: str = "k-nearest-p-median",
    ):
        """
        Create the object of KNearestPMedian class using input data.

        Parameters
        ----------
        gdf_demand : GeoDataFrame
            A GeoDataFrame containing demand points with their associated attributes.
        gdf_fac : GeoDataFrame
            A GeoDataFrame containing facility points with their associated attributes.
        demand_col : str
            The column name in gdf_demand representing the coordinate of each client.
        facility_col : str
            The column name in gdf_fac representing the coordinate of each facility.
        weights_cols : str
            The column name in gdf_demand representing the weights for each client.
        p_facilities: int
           The number of facilities to be located.
        facility_capacity_col : str, optional
            The column name in gdf_fac representing the capacity of each facility,
            by default None.
        k_array : np.array, optional
            An array of integers representing the k values for each client.
            If not provided, a default value of 5 or the number of facilities,
            whichever is smaller, will be used.
        distance_metric : str, optional
            The distance metric to be used in calculating distances between clients
            and facilities, by default "euclidean".
        name : str, optional
            The name of the problem, by default "k-nearest-p-median".

        Returns
        -------

        spopt.locate.p_median.KNearestPMedian

        Examples
        --------
        >>> from spopt.locate import KNearestPMedian
        >>> import geopandas

        Create the input data and attributes.

        >>> k = np.array([1, 1])
        >>> demand_data = {
        ...    'ID': [1, 2],
        ...    'geometry': [Point(0.5, 1), Point(1.5, 1)],
        ...    'demand': [1, 1]}
        >>> facility_data = {
        ...    'ID': [101, 102],
        ...    'geometry': [Point(1,1), Point(0, 2), Point(2, 0)],
        ...    'capacity': [1, 1, 1]}
        >>> gdf_demand = geopandas.GeoDataFrame(demand_data, crs='EPSG:4326')
        >>> gdf_fac = geopandas.GeoDataFrame(facility_data, crs='EPSG:4326')

        Create and solve a ``KNearestPMedian`` instance from the geodataframe.

        >>> k_nearest_pmedian = KNearestPMedian.from_geodataframe(
        ...     gdf_demand, gdf_fac,'geometry','geometry', weights_cols='demand',
        ...     2, facility_capacity_col='capacity', k_array = k)
        >>> k_nearest_pmedian = k_nearest_pmedian.solve(pulp.PULP_CBC_CMD(msg=False))

        Get the facility-client associations.

        >>> for fac, cli in enumerate(k_nearest_pmedian.fac2cli):
        ...     print(f"facility {fac} serving {len(cli)} clients")
        facility 0 serving 1 clients
        facility 1 serving 1 clients
        facility 2 serving 0 clients

        Get the total and average weighted travel cost.

        >>> round(k_nearest_pmedian.problem.objective.value(), 3)
        1.618
        >>> round(k_nearest_pmedian.mean_dist, 3)
        0.809

        Get the k values for the last iteration.
        >>> print(k_nearest_pmedian.k_array)
        [2, 1]

        """

        # check the crs of two geodataframes
        if gdf_demand.crs is None:
            raise ValueError("GeoDataFrame gdf_demand does not have a valid CRS.")
        if gdf_fac.crs is None:
            raise ValueError("GeoDataFrame gdf_facility does not have a valid CRS.")
        if gdf_demand.crs != gdf_fac.crs:
            raise ValueError(
                "Geodataframes crs are different: "
                f"gdf_demand-{gdf_demand.crs}, gdf_fac-{gdf_fac.crs}"
            )

        # create the array of coordinate of clients and facilities
        dem = gdf_demand[demand_col]
        fac = gdf_fac[facility_col]
        dem_data = np.array([dem.x.to_numpy(), dem.y.to_numpy()]).T
        fac_data = np.array([fac.x.to_numpy(), fac.y.to_numpy()]).T

        # check the values of k_array
        if k_array is None:
            k_array = np.full(len(dem_data), np.minimum(len(fac_data), 5))
        elif not isinstance(k_array, np.ndarray):
            raise TypeError("k_array should be a numpy array.")
        elif not (k_array <= len(fac_data)).all():
            raise ValueError(
                f"The value of k should be no more than the number of total "
                f"facilities, which is {len(fac_data)}."
            )

        # demand and capacity
        service_load = gdf_demand[weights_cols].to_numpy()
        weights_sum = service_load.sum()
        facility_capacities = None
        if facility_capacity_col is not None:
            facility_capacities = gdf_fac[facility_capacity_col].to_numpy()

        return KNearestPMedian(
            weights_sum,
            dem_data,
            fac_data,
            service_load,
            k_array,
            p_facilities,
            facility_capacities,
            distance_metric,
            name,
        )

    def facility_client_array(self) -> None:
        """

        Create a 2D array storing **facility to client relationships** where each
        row represents a facility and contains an array of client indices with
        which it is associated. An empty client array indicates the facility
        is associated with no clients.

        Returns
        -------

        None
        """
        fac_vars = getattr(self, "fac_vars")
        cli_vars = getattr(self, "cli_assgn_vars")
        len_fac_vars = len(fac_vars)

        self.fac2cli = []

        for j in range(len_fac_vars):
            array_cli = []
            if fac_vars[j].value() > 0:
                for i in range(len(cli_vars)):
                    if (i, j) in cli_vars and cli_vars[i, j].value() > 0:
                        array_cli.append(i)

            self.fac2cli.append(array_cli)

    def solve(self, solver: pulp.LpSolver, results: bool = True):
        """

        Solve the k nearest p-median model.

        This method iteratively solves the KNearestPMedian model using a specified
        solver until no more clients need to be assigned to placeholder facilities.
        The k values for clients are increased dynamically based on the presence of
        clients not assigned to k nearest facilities.

        Parameters
        ----------
        solver : pulp.LpSolver
            The solver to be used for solving the optimization model.
        results : bool (default True)
            If ``True`` it will create metainfo (which facilities cover which
            demand) and vice-versa, and the uncovered demand.

        Returns
        -------

        spopt.locate.p_median.KNearestPMedian

        """

        # initialize the sum of the values of g_i
        sum_gi = 1

        # start the loop
        while sum_gi > 0:
            self._create_sparse_matrix()
            self._from_sparse_matrix()
            self.problem.solve(solver)
            self.check_status()

            # check the result
            placeholder_vars = getattr(self, "placeholder_vars")
            sum_gi = sum(
                placeholder_vars[i].value()
                for i in range(len(placeholder_vars))
                if placeholder_vars[i].value() > 0
            )
            if sum_gi > 0:
                self._update_k_array()

        if results:
            self.facility_client_array()
            self.client_facility_array()
            self.get_mean_distance()

        return self

## How to Use the New Model

### References:

- [Church, R. L. (2018). Tobler’s Law and Spatial Optimization: Why Bakersfield? International Regional Science Review, 41(3), 287–310.](https://doi.org/10.1177/0160017616650612) 